# Session de Recrutement Printemps 2017 - Team Tax Analytics
## Rendu de Guillaume Grelet - guillaume.grelet@telecom-paristech.fr

Date : février 2017


Ce rendu a pour but de fournir une réponse à l'exercice posé par PwC sur la gestion de tableur Excel.  
Il utilise les packages suivante : jupyter, pandas, PyQt, ipywidgets  
  
  
Notes :
Le code suivant est systématiquement commenté en anglais, comme l'usage le veut.  


In [205]:
# Import the necessary packages..

# For the widgets
%gui qt 
from IPython.display import display, Markdown, Latex
from PyQt5.QtWidgets import QFileDialog
from ipywidgets import *

# For the excel file processing...
import pandas as pd

import jellyfish


ImportError: No module named 'jellyfish'

In [145]:
def filesearch(dir=None):
    """
    That function allows the user to select a file via a PyQt5 dialog window and return the file name as a str
    """
    if dir is None: dir ="~/" # Default path is ~/
    filename = QFileDialog.getOpenFileName(None)
    return filename[0]

In [152]:
datapath = filesearch("~/")

In [147]:
%%time
print("Processing file...")
data = pd.read_excel(datapath)
print("\nProcessing OK. Stats about processing:\n")

Processing file...

Processing OK. Stats about processing:

CPU times: user 9.12 s, sys: 308 ms, total: 9.43 s
Wall time: 9.88 s


In [190]:
input_label_widget = widgets.Textarea(
    placeholder='Place here the preprocessed labels',
    value='code_hospidiag\nsous_axe',
    disabled=False
)

button1 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


display(Markdown("## Remplissez ci-dessus la liste des labels standards préprogrammés  "), input_label_widget, button1)

def handle_submit(sender):
    global x
    x = input_label_widget.value
    global input_label_list
    input_label_list = x.split()
    print("input_label_list = ")
    print(input_label_list)
    return None

def when_button_clicked(button1):
    out = handle_submit(input_label_widget)
    return None 

button1.on_click(when_button_clicked)

## Remplissez ci-dessus la liste des labels standards préprogrammés  

input_label_list = 
['code_hospidiag', 'sous_axe']
input_label_list = 
['code_hospidiag', 'sous_axe', 'test']


In [192]:
found_labels = list(data)
print("Found labels are the following:")
for str in found_labels:
    print("    " + str)

Found labels are the following:
    axe
    sous_axe
    indic
    code_hospidiag
    libelle


In [200]:
intersection = list(set(found_labels).intersection(input_label_list))
#print(intersection)

existing_label_widget = widgets.Textarea(
    value="\n".join(intersection) + "\n",
    placeholder='Type something',
    disabled=True
)

display(Markdown("## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée"), existing_label_widget)

## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée

In [201]:
new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
                
t = widgets.Textarea(
    value="\n".join(new_labels) + "\n",
    placeholder='Type something',
    disabled=False
)

b = widgets.Button(
    description='Ok',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter"), t, b)

def handle_submit(sender):
    global y
    y = t.value
    global modified_list
    modified_list = y.split()
    print("modified_list = ")
    print(modified_list)
    return None

def when_button_clicked(button):
    out = handle_submit(t)
    return None 

b.on_click(when_button_clicked)

## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter

In [138]:
'''md_str = "# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input"
new_label_str = "[\"" + "\", \"".join(new_labels) + "\"]"

input_str = "label_list = input_label_list + " + new_label_str

display(Markdown(md_str))
get_ipython().set_next_input(input_str)'''

['code_hospidiag', 'libelle']